In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from train import setup_dataloaders, setup_experiment, build_env, do_train

In [76]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   87450 KB |  412807 KB |    7745 MB |    7659 MB |
|       from large pool |   74880 KB |  375296 KB |    7505 MB |    7432 MB |
|       from small pool |   12570 KB |   47011 KB |     239 MB |     227 MB |
|---------------------------------------------------------------------------|
| Active memory         |   87450 KB |  412807 KB |    7745 MB |    7659 MB |
|       from large pool |   74880 KB |  375296 KB |    7505 MB |

In [101]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

                         conv1 has       9408 params (~  0.1) %
                           bn1 has        128 params (~  0.0) %
                          relu has          0 params (~  0.0) %
                       maxpool has          0 params (~  0.0) %
                        layer1 has     147968 params (~  1.0) %
                        layer2 has     525568 params (~  3.4) %
                        layer3 has    2099712 params (~ 13.7) %
                        layer4 has    8393728 params (~ 54.9) %
               alg_confidences has    3808529 params (~ 24.9) %
                 deconv_layers has     295104 params (~  1.9) %
                   final_layer has        561 params (~  0.0) %
total params:             15280706
freezing Conv2d
freezing BatchNorm2d
freezing ReLU
freezing MaxPool2d
freezing Sequential
freezing Sequential
freezing Sequential
freezing Sequential
                         conv1 has          0 params (~  0.0) %
                           bn1 has          0 p